Noise Reduction in the Acquired Point Cloud

- 4a: 
    - raw
    - merged
- 4b: 
    - ground segmentation (1000 iteration)
- 4c:
    - filterOBB from YOLO
    - RANSAC
        - RANSAC Global Registration
        - ICP Local Refinement
        - CAD-Guided RANSAC Filtering
- 4d:
    - FPS

Pose estimation
ICP (filtered_pointcloud, reference_CAD)
- initial guess using position and orientation from YOLO OBB


In [1]:
print("Werucome")

Werucome


In [67]:
# Show PCD files from file(s) #

import open3d as o3d
import numpy as np

# 1. Define the list of your PCD file paths

stl_path = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\0_cad_stl\first_object_scaled.stl"
mesh = o3d.io.read_triangle_mesh(stl_path)


pcd_paths = [
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\merged_filtered_transformed_cleaned.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\0_cad_stl\first_object_scaled.pcd"
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\0.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\2_segmented_data\0.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\0.pcd"
]

# Full list of PCD files from 30 degree rotation plus the original helios
pcd_paths = [
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\4_ground_segmented.pcd"
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\0_helios.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\0.pcd", #1
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\1.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\2.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\3.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\4.pcd", 
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\5.pcd", 
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\6.pcd", #2
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\7.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\8.pcd"
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\9.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\10.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\11.pcd",
    # r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\12.pcd" #3
]

# Full list of filtered PCD files
# pcd_paths = [
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\0.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\1.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\2.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\3.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\4.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\5.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\6.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\7.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\8.pcd"
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\9.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\10.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\11.pcd",
#     r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_filtered_data\12.pcd"
# ]



# 2. Define a list of colors (RGB values between 0 and 1), it's circulating
colors = [
    [1, 0, 0],      # Red
    [0, 1, 0],      # Green
    [0, 0, 1],      # Blue 
    [1, 1, 0],      # Yellow
    [1, 0, 1],      # Magenta
    [0, 1, 1],      # Cyan
    [0.5, 0, 0],    # Dark Red
    [0, 0.5, 0],    # Dark Green
    [0, 0, 0.5],    # Dark Blue
    [0.5, 0.5, 0],  # Olive
    [0.5, 0, 0.5],  # Purple
    [0, 0.5, 0.5],  # Teal
    [1, 0.5, 0],    # Orange
    [1, 0, 0.5],    # Pink
    [0.5, 1, 0],    # Lime
    [0, 1, 0.5]     # Aquamarine
]

pcd_list = []

# 3. Loop through the paths, load, and color them
for i, path in enumerate(pcd_paths):
    pcd = o3d.io.read_point_cloud(path)
    
    # Check if the file was loaded correctly
    if not pcd.has_points():
        print(f"Warning: {path} is empty or not found.")
        continue
    
    # Select a color from the list (using modulo % to cycle through colors if paths > colors)
    # color = colors[i % len(colors)]
    # pcd.paint_uniform_color(color)

    # Optional: Shift a bit each point cloud for better visualization
    # pcd.translate((i * 0.001, 0, 0)) # very small shift
    # pcd.translate((i * 0.2, 0, 0)) # very small shift
    
    # Add to the list for visualization
    pcd_list.append(pcd)

# 4. Visualize all point clouds at once
# Origin coordinate frame
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.075, origin=[0, 0, 0]) # x: red y: green z: blue
# pcd_list.append(origin)

# CAD mesh
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh_pcd = mesh.sample_points_uniformly(number_of_points=10000)
# mesh_pcd.paint_uniform_color([0.7, 0.7, 0.7])  # Light gray
pcd_list.append(mesh_pcd)


# Misc
# extra = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0.1]) # x: red y: green z: blue
# pcd_list.append(extra)

if pcd_list:
    o3d.visualization.draw_geometries(pcd_list,
                                      zoom=1, 
                                      front=[0.0, 3, 3],
                                      up=[-0.0, 0.0, 1],
                                      window_name="Multi-view Visualization")

else:
    print("No point clouds were loaded.")

In [44]:
# Merging Raw Data #
# Processing 4a1 to 4a2 #

pcd_paths = [
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\0_helios.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\0.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\1.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\2.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\3.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\4.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\5.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\6.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\7.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\8.pcd"
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\9.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\10.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\11.pcd",
    r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\12.pcd"
]

merged_pointcloud = o3d.geometry.PointCloud()

for path in pcd_paths:
    pcd = o3d.io.read_point_cloud(path)
    merged_pointcloud += pcd

# Visualize merged point cloud
o3d.visualization.draw_geometries([merged_pointcloud], window_name="Merged Point Cloud")

# Save merged point cloud
o3d.io.write_point_cloud(r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\2_merged.pcd", merged_pointcloud)


[Open3D WARNING] Read PCD failed: unable to open file: C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\8.pcdC:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\1_raw_data\30degree\9.pcd


True

In [387]:
# Initial Filtering and Processing PCD Files #
# Processing 4a2 to 4b #

import open3d as o3d
import numpy as np
# import pcl
import gc

def filter_points_by_z(pcd, z_min=0, z_max=0.4):
    # Convert to NumPy array
    points = np.asarray(pcd.points)
    # Filter by Z-axis values
    filtered_points = points[(points[:, 2] > z_min) & (points[:, 2] < z_max)]
    # Create filtered point cloud
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
    return filtered_pcd

def remove_ground_plane(pcd, distance_threshold=0.0035, ransac_n=3, num_iterations=300):
    # Plane segmentation for ground removal
    plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n, num_iterations=num_iterations)
    # Remove ground points
    pcd_without_ground = pcd.select_by_index(inliers, invert=True)
    return pcd_without_ground

def remove_statistical_outlier(pcd, nb_neighbors=20, std_ratio=2.0):
    # Remove outliers
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
    filtered_pcd = pcd.select_by_index(ind)
    return filtered_pcd

# def open3d_to_pcl(open3d_cloud):
#     # Convert Open3D point cloud to PCL point cloud
#     np_points = np.asarray(open3d_cloud.points, dtype=np.float32)
#     pcl_cloud = pcl.PointCloud()
#     pcl_cloud.from_array(np_points)
#     return pcl_cloud

# def pcl_to_open3d(pcl_cloud):
#     # Convert PCL point cloud to Open3D point cloud
#     pcl_array = pcl_cloud.to_array()
#     open3d_cloud = o3d.geometry.PointCloud()
#     open3d_cloud.points = o3d.utility.Vector3dVector(pcl_array)
#     return open3d_cloud

if __name__ == "__main__":
    # List of original PCD file paths
    pcd_paths = [
        r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\2_merged.pcd"
    ]

    # List of save paths for Z-filtered PCD files
    z_filtered_pcd_paths = [
        r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_z_filtered.pcd"
    ]

    # List of save paths for final filtered PCD files
    final_filtered_pcd_paths = [
        r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\4_ground_segmented.pcd"
    ]


    open3d_clouds = []

    # Process each point cloud file individually
    for idx, (raw_path, z_filtered_path, final_filtered_path) in enumerate(zip(pcd_paths, z_filtered_pcd_paths, final_filtered_pcd_paths)):
        print(f"Processing file: {raw_path}")
        pcd = o3d.io.read_point_cloud(raw_path)
        if not pcd.has_points():
            print(f"Warning: Point cloud {raw_path} is empty.")
            continue

        # Z-axis filtering
        z_filtered_pcd = filter_points_by_z(pcd, z_min=0, z_max=0.05)
        o3d.io.write_point_cloud(z_filtered_path, z_filtered_pcd)
        print(f"Saved Z-filtered point cloud: {z_filtered_path}")

        # Ground removal
        try:
            pcd_no_ground = remove_ground_plane(z_filtered_pcd, distance_threshold=0.0035, ransac_n=3, num_iterations=1000)
        except Exception as e:
            print(f"Error occurred during ground removal ({z_filtered_path}): {e}")
            continue

        # Outlier removal
        pcd_filtered = remove_statistical_outlier(pcd_no_ground, nb_neighbors=20, std_ratio=2.0)

        # Save final filtered point cloud
        o3d.io.write_point_cloud(final_filtered_path, pcd_filtered)
        print(f"Saved final filtered point cloud: {final_filtered_path}")

        # Add to list for visualization
        open3d_clouds.append(pcd_filtered)

        # Clear memory
        del pcd
        del z_filtered_pcd
        del pcd_no_ground
        del pcd_filtered

        # Force garbage collection
        gc.collect()

o3d.visualization.draw_geometries(open3d_clouds, window_name="Filtered PointClouds with Colors", width=1024, height=768)


Processing file: C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\2_merged.pcd
Saved Z-filtered point cloud: C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\3_z_filtered.pcd
Saved final filtered point cloud: C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\4_ground_segmented.pcd


In [49]:
# Centering Merged Point Cloud and Manually predict the boundary #

merged_pointcloud_path = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\4_ground_segmented.pcd"

pcd_list = []

merged_pointcloud = o3d.io.read_point_cloud(merged_pointcloud_path)
transformation = np.eye(4)
transformation[:3, 3] = -merged_pointcloud.get_center()
merged_pointcloud.transform(transformation)
pcd_list.append(merged_pointcloud)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[0.0, 0.0, 0]) # x: red y: green z: blue
pcd_list.append(origin)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[0.075, 0.0, 0]) # x: red y: green z: blue
pcd_list.append(origin)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[-0.075, 0.0, 0]) # x: red y: green z: blue
pcd_list.append(origin)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[-0.03, -0.055, 0]) # x: red y: green z: blue
pcd_list.append(origin)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[0.0, 0.06, 0]) # x: red y: green z: blue
pcd_list.append(origin)

o3d.visualization.draw_geometries(pcd_list, window_name="Merged Point Cloud")

o3d.io.write_point_cloud(r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\5_transformed.pcd", merged_pointcloud)





True

In [22]:
# Manually Clean Merged Point Cloud #

import open3d as o3d
import numpy as np

# 1. Load and Center
merged_pointcloud_path = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\5_transformed.pcd" 
merged_pointcloud = o3d.io.read_point_cloud(merged_pointcloud_path)

transformation = np.eye(4)
transformation[:3, 3] = -merged_pointcloud.get_center()
merged_pointcloud.transform(transformation)

# 2. Define Boundary Points
# Ensure these points are in order around the perimeter
boundary_points = np.array([
    [0.075, 0.0, 0],
    [0.0, 0.06, 0],
    [-0.075, 0.0, 0],
    [-0.03, -0.055, 0],
])

# 3. Setup the Selection Polygon Volume
vol = o3d.visualization.SelectionPolygonVolume()

# Use 'z' (lowercase) or 'Z' for the axis
vol.orthogonal_axis = "Z" 
vol.bounding_polygon = o3d.utility.Vector3dVector(boundary_points)

# Use these methods to set the Z-range
vol.axis_max = 0.5   # The "top" of your selection box
vol.axis_min = -0.5  # The "bottom" of your selection box

# 4. Crop
cropped_pcd = vol.crop_point_cloud(merged_pointcloud)

# 5. Build Visualization List
pcd_list = [cropped_pcd]

# origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[0.0, 0.0, 0]) # x: red y: green z: blue
# pcd_list.append(origin)

# Adding your visual markers (coordinate frames)
# for pt in boundary_points:
#     origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.02, origin=pt)
#     pcd_list.append(origin)

o3d.visualization.draw_geometries(pcd_list)

o3d.io.write_point_cloud(r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\6_bounding_boxed.pcd", cropped_pcd)



# test_bentar = o3d.io.read_point_cloud(r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\6_bounding_boxed.pcd")
# test_bentar.transform(np.linalg.inv(transformation))
# o3d.io.write_point_cloud(r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\6_bounding_boxed_untransformed.pcd", test_bentar)

# #visualization
# axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0]) # x: red y: green z: blue
# pcd_list = [test_bentar, axis]
# o3d.visualization.draw_geometries(pcd_list)



True

In [64]:
# RANSAC Global Alignment between Merged PCD and Reference STL #


import open3d as o3d
import numpy as np
import copy


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp])
    
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    source = merged_pcd
    target = ref_pcd
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    o3d.visualization.draw_geometries([source_down])

    return source, target, source_down, target_down, source_fpfh, target_fpfh

merged_pointcloud_path = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\6_bounding_boxed.pcd" 
ref_stl_path        = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\0_cad_stl\first_object_scaled.stl"

ref_mesh = o3d.io.read_triangle_mesh(ref_stl_path)
ref_pcd = ref_mesh.sample_points_uniformly(number_of_points=10000)
merged_pcd = o3d.io.read_point_cloud(merged_pointcloud_path)


voxel_size = 0.003
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)



def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result


result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)


# Apply transformation to original source point cloud using copy data

# output = copy.deepcopy(source)
# output.transform(result_ransac.transformation)
# o3d.visualization.draw_geometries([output, merged_pcd], window_name="Final Alignment Result")

o3d.io.write_point_cloud(r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\7_global_aligned.pcd", output)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.003.
:: Estimate normal with search radius 0.006.
:: Compute FPFH feature with search radius 0.015.
:: Downsample with a voxel size 0.003.
:: Estimate normal with search radius 0.006.
:: Compute FPFH feature with search radius 0.015.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.003,
   we use a liberal distance threshold 0.005.
[Open3D WARNING] Too few correspondences (85) after mutual filter, fall back to original correspondences.
RegistrationResult with fitness=6.242724e-01, inlier_rmse=2.599943e-03, and correspondence_set size of 2145
Access transformation to get result.


True

In [54]:
# ICP Alignment between CAD and Merged PCD #


import open3d as o3d
import numpy as np
import copy

ref_stl_path        = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\0_cad_stl\first_object_scaled.stl"
filtered_pcd_path   = r"C:\Users\Alvan\Documents\Alvan\Data\Code\Python\Main\7_global_aligned.pcd"

ref_mesh = o3d.io.read_triangle_mesh(ref_stl_path)
ref_pcd = ref_mesh.sample_points_uniformly(number_of_points=25000)

filtered_pcd = o3d.io.read_point_cloud(filtered_pcd_path)


origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05, origin=[0, 0, 0]) # x: red y: green z: blue

o3d.visualization.draw_geometries([ref_pcd, filtered_pcd, origin_frame], window_name="Before ICP Alignment")

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])



threshold = 0.0001
trans_init = np.eye(4)
# trans_init[:3, 3] = - (ref_pcd.get_center() - filtered_pcd.get_center())


print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(ref_pcd, filtered_pcd, threshold, trans_init)
print(evaluation)



print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source=filtered_pcd, 
    target=ref_pcd, 
    max_correspondence_distance=threshold,
    init=trans_init,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(relative_rmse=1e-3, max_iteration=20000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(ref_pcd, filtered_pcd, reg_p2p.transformation)

Initial alignment
RegistrationResult with fitness=4.800000e-04, inlier_rmse=7.594200e-05, and correspondence_set size of 12
Access transformation to get result.
Apply point-to-point ICP
RegistrationResult with fitness=6.332650e-04, inlier_rmse=6.534671e-05, and correspondence_set size of 17
Access transformation to get result.
Transformation is:
[[ 9.99999885e-01  4.28334779e-04 -2.15053123e-04 -3.82035653e-06]
 [-4.28780811e-04  9.99997748e-01 -2.07831374e-03  6.36859738e-05]
 [ 2.14162424e-04  2.07840572e-03  9.99997817e-01 -4.85702363e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# FPS #
# Processing 4c2 to 4d #